In [1]:
import numpy as np
import torch.nn as nn

In [ ]:
class yolo_loss(nn.Module):
    def __init__(self):
        super(yolo_loss, self).__init__()

    def forward(self, 
                x_pred, x_actual, y_pred, y_actual,
                w_pred, w_actual, h_pred, h_actual,
                is_obj, no_obj, class_pred, class_actual):
        obj_identity = None
        no_obj_identity = None

        bbox_l2_norm = np.power((x_pred - x_actual), 2) + np.power((y_pred - y_actual), 2)
        bbox_coord_loss = np.sum(np.sum(np.multiply(obj_identity, bbox_l2_norm)))

        pass